In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
from tabulate import tabulate
import time
from datetime import date, timedelta
import win32com.client as win32
import teradatasql
import openpyxl
import matplotlib.pyplot as plt
import telebot
import logging
import types


In [7]:
user=input('ввести логин терадаты')
password=input("пароль от терадаты")
query = '''select * from uat_ca.vf_for_tg_test'''
with teradatasql.connect(host="td2800.corp.tele2.ru", user=user, password=password,logmech="ldap", tmode="TERA" ) as connect:
    df = pd.read_sql(query, connect)

C:\Users\vitaly.flerin\AppData\Local\Temp\ipykernel_8276\3506826681.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connect)


подготовка

In [9]:
token = '5859647042:AAE0VTK8RlKrhFx7Mv9HeJS3tFK7aZ_J4W4'
your_chat_id = '322276469'
chat_id = '322276469'
bot = telebot.TeleBot(token)
#Даша =1482693734

бэкэнд


In [ ]:

from telegram import ReplyKeyboardMarkup, KeyboardButton

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    if message.from_user.id != bot.get_me().id:
        markup = ReplyKeyboardMarkup(row_width=2)
        item1 = KeyboardButton('/hello')
        item2 = KeyboardButton('/plot')
        item2 = KeyboardButton('/inf')
        markup.add(item1, item2)
        bot.send_message(chat_id=message.chat.id, text="Привет!", reply_markup=markup)

@bot.message_handler(commands=['hello', 'inf'])
def send_hello(message):
    if message.from_user.id != bot.get_me().id:
        t = "Good day to you too!" # Заменили данные на сообщение приветствия
        t_str = str(t)
        # Send the message to the user
        bot.send_message(chat_id=message.chat.id, text=t_str)

@bot.message_handler(commands=['plot'])
def send_plot(message):
    if message.from_user.id != bot.get_me().id:
        # Создаем данные для графика       
# построение графика
        df['create_month'] = pd.to_datetime(df['create_month'])
        df['year'] = df['create_month'].dt.year
        df['month'] = df['create_month'].dt.month
        df_pivot = df.pivot_table(index='month', columns='year', values='nps')
        df_pivot.plot()
        # настройка осей и легенды
        plt.xlabel('Month')
        plt.ylabel('NPS')
        plt.title('NPS by Month and Year')
        plt.legend(title='Year')
        # Сохраняем график в объект io.BytesIO
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0) 
        # Отправляем график через бота
        bot.send_photo(chat_id=message.chat.id, photo=plot_object)
# Запускаем процесс диспетчеризации
bot.polling()

Other

In [ ]:
#https://api.telegram.org/bot5859647042:AAE0VTK8RlKrhFx7Mv9HeJS3tFK7aZ_J4W4/getUpdates
# как узнать свой чат ай ди, - пройти по ссылке
bot = telebot.TeleBot("5859647042:AAE0VTK8RlKrhFx7Mv9HeJS3tFK7aZ_J4W4")
# Получаем список обновлений
updates = bot.get_updates()
# Получаем последнее обновление
last_update = updates[-1]
# Получаем chat_id из последнего обновления
chat_id = last_update.message.chat.id
print(chat_id)

самостоятельная рассылка

In [11]:
#просто текст
#отправлем сообщение всем подписчикам
#Получаем список всех подписчиков бота
subscribers = bot.get_updates()
test = 'Hello, me!'
# Отправляем сообщение себе
bot.send_message(chat_id=your_chat_id, text=test )
# Отправляем сообщение каждому подписчику
for subscriber in subscribers:
    if subscriber and subscriber.message and subscriber.message.chat:
        user_id = subscriber.message.chat.id
        bot.send_message(chat_id=user_id, text=test)